In [3]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

 ········


In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 24, 'total_tokens': 28, 'completion_time': 0.003333333, 'prompt_time': 0.003105405, 'queue_time': 0.011489543, 'total_time': 0.006438738}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-5e11930d-2da4-4013-bd53-c995416b63f4-0', usage_metadata={'input_tokens': 24, 'output_tokens': 4, 'total_tokens': 28})

In [8]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="")])
response.content

'According to various dictionaries, a question is defined as:\n\n* "A sentence or phrase that asks for information or an explanation" (Merriam-Webster)\n* "A sentence or phrase that asks for information, requires a response, or expresses uncertainty or inquiry" (Oxford English Dictionary)\n* "A sentence or phrase that is used to ask for information, advice, or an explanation, and is typically followed by a verb in the form of \'What?\', \'Why?\', \'How?\', etc." (Cambridge Dictionary)\n\nIn general, a question is a linguistic expression that seeks information, clarification, or an answer from someone or something. It is typically marked by a verb in the interrogative form (e.g., "What?", "Why?", "How?", etc.) and usually ends with a question mark (?) or an exclamation mark (!). Questions can be asked in various forms, including written or spoken language, and are an essential part of communication and learning.'

In [11]:
!pip install -qU langchain-community wikipedia

In [14]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(tool.invoke({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.


In [13]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from pydantic import BaseModel, Field


class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia, should be 3 or less words"
    )


tool = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

print(tool.run("langchain"))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


In [15]:
print(f"Name: {tool.name}")
print(f"Description: {tool.description}")
print(f"args schema: {tool.args}")
print(f"returns directly?: {tool.return_direct}")

Name: wikipedia
Description: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
args schema: {'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}
returns directly?: False


In [34]:
import os

from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper

os.environ["SERPAPI_API_KEY"] = ""
tool = GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper())

In [35]:
tool.run("Google")

'\nQuery: Google\nstock: GOOG:NASDAQ\nprice: $173.82\npercentage: 1.99\nmovement: Down\nus: price = 43440.23, movement = Down\neurope: price = 19220.03, movement = Down\nasia: price = 38642.91, movement = Up\n'

In [36]:
import os

from langchain.agents import AgentType, initialize_agent, load_tools

In [44]:
llm = ChatGroq(model="llama3-8b-8192")
tools = load_tools(["google-finance"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
)
agent.run("When should I invest in Apple?")



> Entering new AgentExecutor chain...
Let's dive in!

Thought: I should try to gather some information about Apple's current stock price and performance before making an investment decision.

Action: google_finance
Action Input: "Apple stock price"
Observation: 
Query: Apple stock price
No summary information
us: price = 43427.22, movement = Down
europe: price = 19220.03, movement = Down
asia: price = 38642.91, movement = Up

Thought:Thought: Okay, it seems like Apple's stock price is currently experiencing a mixed performance, with a decline in the US and Europe, but an increase in Asia. This could be due to various factors such as market trends, company performance, and global events.

Action: google_finance
Action Input: "Apple stock performance"
Observation: 
Query: Apple stock performance
No summary information
us: price = 43427.22, movement = Down
europe: price = 19220.03, movement = Down
asia: price = 38642.91, movement = Up

Thought:Thought: It seems that my previous observat

"It seems that Apple's stock price is experiencing a mixed performance, with a consistent decline in the US and Europe, and an increase in Asia. The earnings report does not provide any information that would suggest when to invest in Apple. However, considering the current trend, it might be worth considering investing in Apple's Asian market, as the stock price is increasing there."